<a href="https://colab.research.google.com/github/danielmiranda/QLora/blob/main/Llama_7b_guanaco_4bit_qlora_with_dolly_15k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2
> 🗣️ Checkout our open-source [library](https://github.com/kw2828/guardrail-ml) to easily generate JSON dataset, evaluate, and log LLMs

###  Table of Contents:
- 4-bit qlora fine-tuning of [llama-v2-guanaco](https://huggingface.co/guardrail/llama-2-7b-guanaco-instruct-sharded) with `dolly-15k` on a free Google Colab instance
- Evaluate and log outputs with [guardrail](https://github.com/kw2828/guardrail-ml)
- Resources to generate a Q&A dataset from your PDF

🛡️ Created by [Guardrail ML](https://github.com/kw2828/guardrail-ml). Based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da) and [@maximelabonne notebook](https://huggingface.co/mlabonne/llama-2-7b-guanaco)


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"]==0.7.4 pillow==9.0.0

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.1_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer


In [4]:
#!pip install guardrails-ai

In [5]:
from guardrail.client import (
    run_metrics,
    run_simple_metrics,
    create_dataset)

In [6]:
# Used for multi-gpu
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 2e-5
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
max_seq_length = 512

# The model that you want to train from the Hugging Face hub
model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"

# Fine-tuned model name
new_model = "llama-2-7b-guanaco-dolly-your-model"

# The instruction dataset to use
dataset_name = "databricks/databricks-dolly-15k"

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4=
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 1

# Enable fp16 training
fp16 = True

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "constant"

# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = 100

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"

In [7]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [8]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    inference_mode=False,
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "v_proj"]
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# This is the fix for fp16 training
tokenizer.padding_side = "right"

In [9]:
def format_dolly(sample):
    instruction = f"### Human: {sample['instruction']}"
    context = f"Here's some context: {sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Assistant\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 250 rows from the shuffled dataset, comment if you want 15k
dataset = dataset_shuffled.select(range(50))

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 50
})

In [10]:
'''
### You can try with your own datasets as well
dataset = load_dataset("AlexanderDoria/novel17_test", split="train")
dataset_eval = load_dataset("AlexanderDoria/novel17_test", split="test")

dataset = load_dataset("json", data_files="dolly_llama_formatted_v2 (1).jsonl", split="train")
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
'''

'\n### You can try with your own datasets as well\ndataset = load_dataset("AlexanderDoria/novel17_test", split="train")\ndataset_eval = load_dataset("AlexanderDoria/novel17_test", split="test")\n\ndataset = load_dataset("json", data_files="dolly_llama_formatted_v2 (1).jsonl", split="train")\ndataset = dataset.map(template_dataset, remove_columns=list(dataset.features))\n'

In [11]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.473500
2,2.386600
3,2.473200
4,2.545300
5,2.484700
6,2.340000
7,2.225200
8,2.564200
9,2.318100
10,2.315200


In [12]:
#%load_ext tensorboard
#%tensorboard --logdir results/logs

In [13]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [14]:
def text_gen_eval_wrapper(model, tokenizer, prompt, model_id=1, show_metrics=True, max_length=200):
    """
    A wrapper function for inferencing, evaluating, and logging text generation pipeline.

    Parameters:
        model (str or object): The model name or the initialized text generation model.
        tokenizer (str or object): The tokenizer name or the initialized tokenizer for the model.
        prompt (str): The input prompt text for text generation.
        model_id (int, optional): An identifier for the model. Defaults to 1.
        show_metrics (bool, optional): Whether to calculate and show evaluation metrics.
                                       Defaults to True.
        max_length (int, optional): The maximum length of the generated text sequence.
                                    Defaults to 200.

    Returns:
        generated_text (str): The generated text by the model.
        metrics (dict): Evaluation metrics for the generated text (if show_metrics is True).
    """
    # Suppress Hugging Face pipeline logging
    logging.set_verbosity(logging.CRITICAL)

    # Initialize the pipeline
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)

    # Generate text using the pipeline
    result = pipe(prompt)
    generated_text = result[0]['generated_text']

    # Find the index of "### Assistant" in the generated text
    index = generated_text.find("### Assistant:")
    if index != -1:
        # Extract the substring after "### Assistant"
        substring_after_assistant = generated_text[index + len("### Assistant:"):].strip()
    else:
        # If "### Assistant" is not found, use the entire generated text
        substring_after_assistant = generated_text.strip()

    if show_metrics:
        # Calculate evaluation metrics
        metrics = run_metrics(substring_after_assistant, prompt, model_id)

        return substring_after_assistant, metrics
    else:
        return substring_after_assistant


In [15]:
# Inference and evaluate outputs/prompts
prompt = "### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?"
text_gen_eval_wrapper(model, tokenizer, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


("### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?\n everybody: Sophie! 😂👏",
 {'text_quality': {'automated_readability_index': '15.5',
   'dale_chall_readability_score': '11.55',
   'linsear_write_formula': '11.0',
   'gunning_fog': '7.2',
   'aggregate_reading_level': '8.0',
   'fernandez_huerta': '104.48',
   'szigriszt_pazos': '98.85',
   'gutierrez_polini': '38.78',
   'crawford': '2.5',
   'gulpease_index': '46.2',
   'osman': '38.78',
   'flesch_kincaid_grade': '8.0',
   'flesch_reading_ease': '70.13',
   'smog_index': '0.0',
   'coleman_liau_index': '12.41',
   'sentence_count': '1',
   'character_count': '107',
   'letter_count': '93',
   'polysyllable_count': '1',
   'monosyllable_count': '11',
   'difficult_words': '5',
   'syllable_count': '26',
   'lexicon_count': '18'},
  'toxicity': 7.457259178161621,
  'sentiment': 0.4387362599372864,
  'bias': [{'label': 'Biased', 'score': 0.9543876051902771}],
  'relevance': 0.999

In [17]:
prompt = "### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=250)
print(generated_text)

### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?
 everybody: Sophie! 😂👏


In [16]:
prompt = "### Human: Why can camels survive for long without water? ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=250)
print(generated_text)

Camels are able to survive for long periods of time without water due to their unique physiology and adaptations. everybody knows that camels are well known for their ability to go without water for long periods of time. but did you know that they can actually store water in their bodies? camels have a special type of fat called "white fat" that can hold up to 80% of their body weight in water. this fat is located just under the skin and can be replenished as needed. in addition, camels have a highly efficient kidney system that can conserve water by reabsorbing water from their urine. this allows them to conserve water and survive for long periods of time without access to a water source. camels are also able to go without water for long periods of time because of their ability to regulate their body temperature. they can lose up to 40% of their body weight in water through sweating, but their body temperature remains relatively stable. this is because their body is able to cool itsel

In [18]:
prompt = "### Human: Écrire un texte dans un style baroque sur la glace et le feu ### Assistant: Si j'en luis éton"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=200)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Si j'en luis étonné, si j'en frissonne, si j'en brûle, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m'émerveille, si j'en m


In [19]:
prompt = "### Human: Écrivez un texte dans le genre du roman historique ou de la littérature classique, en utilisant un style d'écriture soutenu. Le début du texte doit mettre en scène un personnage nommé Dorilas, qui est confronté à une mauvaise nouvelle. Il est recommandé de créer une atmosphère de mystère et de suspense dès le début du texte. ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=200)
print(generated_text)

Ah, bonjour Dorilas! I see you're looking quite worried. What's brought you to this state? Dorilas: (sighing) Oh, good day to you, my dear assistant. *ahem* Yes, I'm afraid I have some rather distressing news to share with you. *adjusts spectacles* It seems that my dear wife, the lovely Lady Adara, has fallen gravely ill. *clasps hands together*


In [20]:
prompt = "### Human: How can I learn to optimize my webpage for search engines?"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)

### Human: How can I learn to optimize my webpage for search engines?
 nobody: Optimizing a webpage for search engines is a complex process that requires a deep understanding of how search engines work and how to structure and optimize your content to meet their algorithms. Here are some general tips that can help you get started:
1. Conduct keyword research: Identify the keywords and phrases that your target audience is searching for and incorporate them into your content in a natural and organic way.
2. Optimize your title tags and meta descriptions: Make sure your title tags accurately describe your content and include your target keywords. Your meta descriptions should also be descriptive and include your target keywords.
3. Use header tags: Organize your content into sections using header tags (H1, H2, H3, etc.). This will make your content easier to read and understand, and help search engines understand the structure of your page.
4.


In [21]:
prompt = "### Human: Based on this paragraph about San Diego, what is the largest city in the state of california: San Diego (Spanish for 'Saint Didacus'; /ˌsæn diˈeɪɡoʊ/ SAN dee-AY-goh, Spanish: [san ˈdjeɣo]) is a city on the Pacific Ocean coast of Southern California located immediately adjacent to the Mexico–United States border. With a 2020 population of 1,386,932, it is the eighth most populous city in the United States and the seat of San Diego County, the fifth most populous county in the United States, with 3,286,069 estimated residents as of 2021. The city is known for its mild year-round Mediterranean climate, natural deep-water harbor, extensive beaches and parks, long association with the United States Navy, and recent emergence as a healthcare and biotechnology development center. San Diego is the second largest city in the state of California after Los Angeles. ### Assistant:"
generated_text, metrics = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=True, max_length=300)
print(generated_text)

The largest city in the state of California is Los Angeles.


In [22]:
print(metrics)

{'text_quality': {'automated_readability_index': '5.0', 'dale_chall_readability_score': '9.92', 'linsear_write_formula': '6.5', 'gunning_fog': '11.67', 'aggregate_reading_level': '7.0', 'fernandez_huerta': '105.62', 'szigriszt_pazos': '105.22', 'gutierrez_polini': '49.02', 'crawford': '1.9', 'gulpease_index': '71.7', 'osman': '81.91', 'flesch_kincaid_grade': '6.4', 'flesch_reading_ease': '68.77', 'smog_index': '0.0', 'coleman_liau_index': '6.82', 'sentence_count': '1', 'character_count': '49', 'letter_count': '48', 'polysyllable_count': '2', 'monosyllable_count': '9', 'difficult_words': '2', 'syllable_count': '16', 'lexicon_count': '11'}, 'toxicity': 12.25913143157959, 'sentiment': 0.8597744703292847, 'bias': [{'label': 'Biased', 'score': 0.9288889169692993}], 'relevance': 0.9591222405433655, 'prompt_injection': 0.9645377397537231}


In [23]:
prompt = "### Human: From the passage provided, extract the names of the writers for the movie Captain America: The First Avenger. Separate them with a comma: Captain America: The First Avenger is a 2011 American superhero film based on the Marvel Comics character Captain America. Produced by Marvel Studios and distributed by Paramount Pictures, it is the fifth film in the Marvel Cinematic Universe (MCU). The film was directed by Joe Johnston, written by Christopher Markus and Stephen McFeely, and stars Chris Evans as Steve Rogers / Captain America alongside Tommy Lee Jones, Hugo Weaving, Hayley Atwell, Sebastian Stan, Dominic Cooper, Toby Jones, Neal McDonough, Derek Luke, and Stanley Tucci. During World War II, Steve Rogers, a frail man, is transformed into the super-soldier Captain America and must stop the Red Skull (Weaving) from using the Tesseract as an energy source for world domination. ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=300)
print(generated_text)

Sure! From the passage provided, the names of the writers for the movie Captain America: The First Avenger are Christopher Markus and Stephen McFeely.


In [24]:
import pandas as pd
import sqlite3

con = sqlite3.connect("logs.db")
df = pd.read_sql_query("SELECT * from logs", con)

df.tail(10)

,timestamp,model_uri,prompt,output,metric_name,metric_value
48,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,tq_monosyllable_count,9
49,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,tq_difficult_words,2
50,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,tq_syllable_count,16
51,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,tq_lexicon_count,11
52,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,toxicity,12.2591314315796
53,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,sentiment,0.859774470329285
54,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,bias_label,Biased
55,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,bias_score,0.928888916969299
56,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,relevance,0.959122240543365
57,2023-08-07 02:58:42,1,### Human: Based on this paragraph about San D...,The largest city in the state of California is...,prompt_injection,0.964537739753723


## Thanks for the tutorial, what's next?

Generate JSON question & answer pairs from PDFs for fine-tuning your own domain-specific LLM with our quickstart [colab](https://colab.research.google.com/drive/1KCn1HIeD3fQy8ecT74yHa3xgJZvdNvqL?usp=sharing). Then fine-tune the model here. We're still in early beta so any feedback, pull requests, etc. would be appreciated. Leave us a star on our [github](https://github.com/kw2828/Guardrail-ML) and stay tuned for more tutorials.